In [1]:
from z3 import Solver, Real, sat

def calc_value_added_tax(output_tax_val: float, input_tax_val: float):
    s = Solver()

    output_tax_amount = Real('output_tax_amount')
    input_tax_amount  = Real('input_tax_amount')
    final_tax_amount  = Real('final_tax_amount')

    # 建立 constraints
    s.add(output_tax_amount == output_tax_val)
    s.add(input_tax_amount  == input_tax_val)
    s.add(final_tax_amount  == output_tax_amount - input_tax_amount)

    # 用 z3.sat 做檢查
    if s.check() == sat:
        m = s.model()
        # 先取得解，轉換成 float，再四捨五入至整數
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return int(round(tax_value))
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def calc_non_value_added_tax(sales_val: float, category: int):
    """
    依照「非加值型營業稅」邏輯計算應納稅額，
    並以四捨五入至整數後回傳。
    
    參數：
      sales_val: 銷售額
      category:  傳入 1~8 的類別代號，對應不同稅率 (TAX_RATE_MAP)
        1.小規模營業人及其他經財政部規定免予申報銷售額之營業人：1%
        2.保險業之再保費收入：1%
        3.銀行業、保險業、信託投資業、證券業、期貨業、票券業及典當業經營專屬本業之銷售額之收入（不含銀行業、保險業經營銀行、保險本業收入）：2%
        4.銀行業、保險業經營銀行、保險本業之收入：5%
        5.銀行業、保險業、信託投資業、證券業、期貨業、票券業及典當業經營非專屬本業之銷售額之收入：5%
        6.夜總會、有娛樂節目之餐飲店：15%
        7.酒家及有陪侍服務之茶室、咖啡廳、酒吧等：25%
        8.農產品批發市場之承銷人及銷售農產品之小規模營業人：0.1%
      
    回傳：
      final_tax_amount (int)：應納營業稅額（四捨五入後）
    """
    TAX_RATE_MAP = {
        1: 0.01,   # 1%
        2: 0.01,   # 1%
        3: 0.02,   # 2%
        4: 0.05,   # 5%
        5: 0.05,   # 5%
        6: 0.15,   # 15%
        7: 0.25,   # 25%
        8: 0.001   # 0.1% => 0.001
    }
    if category not in TAX_RATE_MAP:
        raise ValueError(f"類別代號 {category} 不在稅率對應表中，請確認輸入是否正確。")
    tax_rate_val = TAX_RATE_MAP[category]

    s = Solver()

    sales_amount = Real('sales_amount')
    tax_rate = Real('tax_rate')
    final_tax_amount = Real('final_tax_amount')

    s.add(sales_amount == sales_val)
    s.add(tax_rate == tax_rate_val)
    s.add(final_tax_amount == sales_amount * tax_rate)

    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return int(round(tax_value))
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


if __name__ == "__main__":
    # ============ 範例 1：加值型營業稅 =============
    # 範例：銷項稅額 = 15000 * 5% = 750 元
    #       進項稅額 = 10000 * 5% = 500 元
    #       應納稅額 = 750 - 500 = 250 元
    vat_example = calc_value_added_tax(output_tax_val=750, input_tax_val=500)
    print(f"[加值型營業稅] 應納(溢付)稅額：{vat_example} 元")

    # 以第 1 類為例 (小規模營業人，稅率=1%)，銷售額=100000
    category_1_example = calc_non_value_added_tax(sales_val=100000, category=1)
    print(f"[非加值型營業稅] 第1類, 銷售額=100000, 應納稅額: {category_1_example} 元")

    # 以第 8 類為例 (農產品批發市場之承銷人 & 小規模營業人，稅率=0.1%)，銷售額=100000
    category_8_example = calc_non_value_added_tax(sales_val=100000, category=8)
    print(f"[非加值型營業稅] 第8類, 銷售額=100000, 應納稅額: {category_8_example} 元")

[加值型營業稅] 應納(溢付)稅額：250 元
[非加值型營業稅] 第1類, 銷售額=100000, 應納稅額: 1000 元
[非加值型營業稅] 第8類, 銷售額=100000, 應納稅額: 100 元


In [4]:
import random
import time
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from z3 import Solver, Real, sat

def round_half_up(x: float) -> int:
    """
    正常四捨五入函式：
      當小數部分 >= 0.5 時，往上取整，
      適用於非負數情形（稅額通常為正）。
    """
    return int(math.floor(x + 0.5))

def calc_value_added_tax(output_tax_val: float, input_tax_val: float):
    s = Solver()

    output_tax_amount = Real('output_tax_amount')
    input_tax_amount  = Real('input_tax_amount')
    final_tax_amount  = Real('final_tax_amount')

    # 建立 constraints
    s.add(output_tax_amount == output_tax_val)
    s.add(input_tax_amount  == input_tax_val)
    s.add(final_tax_amount  == output_tax_amount - input_tax_amount)

    # 用 z3.sat 做檢查
    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        # 使用自訂的正常四捨五入函式
        return round_half_up(tax_value)
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def calc_non_value_added_tax(sales_val: float, category: int):
    """
    依照「非加值型營業稅」邏輯計算應納稅額，
    並以正常四捨五入（xxxx.5時進位）後回傳整數結果。
    
    參數：
      sales_val: 銷售額
      category:  傳入 1~8 的類別代號，對應不同稅率 (TAX_RATE_MAP)
    """
    TAX_RATE_MAP = {
        1: 0.01,   # 1%
        2: 0.01,   # 1%
        3: 0.02,   # 2%
        4: 0.05,   # 5%
        5: 0.05,   # 5%
        6: 0.15,   # 15%
        7: 0.25,   # 25%
        8: 0.001   # 0.1% => 0.001
    }
    if category not in TAX_RATE_MAP:
        raise ValueError(f"類別代號 {category} 不在稅率對應表中，請確認輸入是否正確。")
    tax_rate_val = TAX_RATE_MAP[category]

    s = Solver()

    sales_amount = Real('sales_amount')
    tax_rate = Real('tax_rate')
    final_tax_amount = Real('final_tax_amount')

    s.add(sales_amount == sales_val)
    s.add(tax_rate == tax_rate_val)
    s.add(final_tax_amount == sales_amount * tax_rate)

    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return round_half_up(tax_value)
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def run_test():

    for i in range(5):
        print(f"===== 迴圈第 {i+1} 次 =====")
        # -------- 加值型營業稅測試 --------
        output_tax_val = int(round(random.uniform(100, 10000)))
        input_tax_val = int(round(random.uniform(100, 10000)))
        
        local_vat = calc_value_added_tax(output_tax_val, input_tax_val)
        print(f"第{i}次，Local VAT (加值型): {local_vat}, Params: {{'output_tax_val': {output_tax_val}, 'input_tax_val': {input_tax_val}}}")
        
          
        # -------- 非加值型營業稅測試 --------
        sales_val = int(round(random.uniform(10000, 1000000)))
        category = random.randint(1, 8)
        
        local_nonvat = calc_non_value_added_tax(sales_val, category)
        print(f"第{i}次，Local Non-VAT (非加值型): {local_nonvat}, Params: {{'sales_val': {sales_val}, 'category': {category}}}")

      

if __name__ == "__main__":
    run_test()


===== 迴圈第 1 次 =====
第0次，Local VAT (加值型): 2445, Params: {'output_tax_val': 4694, 'input_tax_val': 2249}
第0次，Local Non-VAT (非加值型): 403, Params: {'sales_val': 402572, 'category': 8}
===== 迴圈第 2 次 =====
第1次，Local VAT (加值型): -1088, Params: {'output_tax_val': 3370, 'input_tax_val': 4458}
第1次，Local Non-VAT (非加值型): 941, Params: {'sales_val': 940567, 'category': 8}
===== 迴圈第 3 次 =====
第2次，Local VAT (加值型): 2673, Params: {'output_tax_val': 6181, 'input_tax_val': 3508}
第2次，Local Non-VAT (非加值型): 4690, Params: {'sales_val': 469002, 'category': 1}
===== 迴圈第 4 次 =====
第3次，Local VAT (加值型): 7355, Params: {'output_tax_val': 7455, 'input_tax_val': 100}
第3次，Local Non-VAT (非加值型): 22300, Params: {'sales_val': 446005, 'category': 4}
===== 迴圈第 5 次 =====
第4次，Local VAT (加值型): 3775, Params: {'output_tax_val': 5977, 'input_tax_val': 2202}
第4次，Local Non-VAT (非加值型): 3749, Params: {'sales_val': 74976, 'category': 4}
